In [62]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import tree
import random
import collections
import re
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Nancy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [22]:
#load data
resp=pd.read_csv("young-people-survey/responses.csv")
podcasts=pd.read_csv("young-people-survey/df_popular_podcasts.csv")
movies=pd.read_csv("young-people-survey/MovieGenre.csv")
music=pd.read_csv("young-people-survey/SpotifyFeatures.csv")
genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [28]:
resp = pd.read_csv("young-people-survey/responses.csv")
podcasts=pd.read_csv("young-people-survey/df_popular_podcasts.csv")
word_exp=re.compile("[^\x00-\x7F]+")
non_eng_pod=[index for index,value in enumerate(list(podcasts["Name"].to_dict().values())) if len(re.findall(word_exp,value))!=0]
podcasts=podcasts.drop(non_eng_pod).drop_duplicates("Name", keep="first")

movies=pd.read_csv("young-people-survey/MovieGenre.csv")
movies=movies.drop_duplicates("imdbId")
movies=movies[movies["IMDB Score"].astype(float)>float(5.0)]

music=pd.read_csv("young-people-survey/SpotifyFeatures.csv")
music=music.drop_duplicates("track_id")
music=music[music["popularity"].astype(float)>float(50)]

genre_IDs=[['1311', 'News & Politics'], ['26', 'Podcasts'], ['1479', 'Social Sciences'], ['1315', 'Science & Medicine'], ['1324', 'Society & Culture'], ['1302', 'Personal Journals'], ['1469', 'Language Courses'], ['1304', 'Education'], ['1320', 'Places & Travel'], ['1416', 'Higher Education'], ['1465', 'Professional'], ['1316', 'Sports & Recreation'], ['1303', 'Comedy'], ['1305', 'Kids & Family'], ['1439', 'Christianity'], ['1314', 'Religion & Spirituality'], ['1444', 'Spirituality'], ['1309', 'TV & Film'], ['1462', 'History'], ['1310', 'Music'], ['1478', 'Medicine'], ['1321', 'Business'], ['1412', 'Investing'], ['1420', 'Self-Help'], ['1307', 'Health'], ['1481', 'Alternative Health'], ['1417', 'Fitness & Nutrition'], ['1467', 'Amateur'], ['1480', 'Software How-To'], ['1318', 'Technology'], ['1448', 'Tech News'], ['1456', 'Outdoor'], ['1477', 'Natural Sciences'], ['1301', 'Arts'], ['1454', 'Automotive'], ['1323', 'Games & Hobbies'], ['1438', 'Buddhism'], ['1443', 'Philosophy'], ['1401', 'Literature'], ['1402', 'Design'], ['1410', 'Careers'], ['1470', 'Training'], ['1413', 'Management & Marketing'], ['1306', 'Food'], ['1406', 'Visual Arts'], ['1446', 'Gadgets'], ['1468', 'Educational Technology'], ['1405', 'Performing Arts'], ['1460', 'Hobbies'], ['1471', 'Business News'], ['1404', 'Video Games'], ['1450', 'Podcasting'], ['1473', 'National'], ['1325', 'Government & Organizations'], ['1461', 'Other Games'], ['1466', 'College & High School'], ['1459', 'Fashion & Beauty'], ['1476', 'Non-Profit'], ['1415', 'K-12'], ['1455', 'Aviation'], ['1464', 'Other'], ['1421', 'Sexuality'], ['1472', 'Shopping'], ['1475', 'Local'], ['1441', 'Judaism'], ['1440', 'Islam'], ['1474', 'Regional'], ['1463', 'Hinduism']]

In [210]:
podcasts.iloc[8110][:]['Name']

'Geeks Gadgets and Guns podcast'

In [30]:
# Better organize the id vs name of genre
dict_pod={int(id[0]):id[1] for id in genre_IDs }

# {Genre ID:NAMES OF PODCAST}
genre_to_podcast={key:[] for key in dict_pod }
for item in podcasts.index:
    array=podcasts["Genre IDs"][item][1:-1].split(',')
    array=[int(array[i].replace('\'', '').strip('\'')) for i in range(len(array))]
    for j in range(len(array)):
        genre_to_podcast[array[j]].append((podcasts["Name"][item], podcasts["Description"][item], podcasts["Podcast URL"][item]))
final_dict={list(dict_pod.values())[p]:list(genre_to_podcast.values())[p] for p in range(len(genre_to_podcast.keys()))}
final_dict = dict((k.lower(), v) for k, v in final_dict.items())

#MAKE MOVIE DICTIONARY
genre_to_movie={}
for i in range( len(movies)):
    line=str(movies.iloc[i]["Genre"])
    for genre in line.split("|"):
        genre=genre.lower()
        if genre in genre_to_movie:
            genre_to_movie[genre].append(movies.iloc[i]["Title"])
        else:
            genre_to_movie[genre]=[movies.iloc[i]["Title"]]

#MAKE MUSIC DICTIONARY
genre_to_music={}
for i in range(len(music)):
    genre=music.iloc[i]['genre']
    genre=genre.lower()
    if genre in genre_to_music:
            genre_to_music[genre].append((music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name']))
    else:
            genre_to_music[genre]=[(music.iloc[i]['track_id'], music.iloc[i]['track_name'],  music.iloc[i]['genre'],  music.iloc[i]['artist_name'])]


In [37]:
song=recs(genre_to_music, ["Folk"])

In [8]:
song=[{
            'title': item[0][1],
            'url': "https://open.spotify.com/embed/track/"+item[0][0],
            'artist': item[0][3], 
            'genre': item[0][2]
            } for item in song]

In [40]:
podcast=recs(final_dict,["News & Politics"])

In [41]:
podcast=[{
            'title': item[0][0],
            'description': item[0][1],
            'url': item[0][2]
            } for item in podcast]

In [175]:
def cosine_sim(corpus):
    vectorizer=TfidfVectorizer(stop_words="english", min_df=1)
    tfidf = vectorizer.fit_transform(corpus)
    similarity = tfidf * tfidf.T
    return similarity.toarray()

In [126]:
cosine_sim("I'd like an apple",  "An apple a day keeps the doctor away")

0.19431434016858146

In [234]:
def get_max_val(np_array): 
    index_max_val=np.argmax(np_array)
    output=(podcasts.iloc[index_max_val][:]["Name"], podcasts.iloc[index_max_val][:]["Description"], podcasts.iloc[index_max_val][:]["Podcast URL"], np_array[index_max_val])
    np_array[index_max_val]=0
    return output

def podcast_recs(query):
    descriptions=list(podcasts['Description'])
    corpus=[query]+descriptions
    matrix=cosine_sim(corpus)
    matrix_slice=matrix[:][0][1:]
    result=[]
    for x in range(5):
        result.append(get_max_val(matrix_slice))
    return result

In [235]:
podcast=podcast_recs("Soft and soothing")

In [236]:
podcast=[{
            'title': item[0],
            'description': item[1],
            'url': item[2], 
            'score': item[3]
            } for item in podcast]

In [237]:
podcast

[{'description': 'Soothing nature sounds to help you to relax and fall asleep.  Bonus episodes include behind-the-scenes information and stories about specific recorded sounds, tips for making your own recordings, listener feedback, and more.',
  'score': 0.21786942662253198,
  'title': 'Sleep with Silk: Nature Sounds (to help insomnia, anxiety, stress, relax, focus, meditate, ASMR)',
  'url': 'https://sleepwithsilk.com'},
 {'description': 'It takes more than great code to be a great engineer. Soft Skills Engineering is a weekly question and answer podcast where software developer hosts answer questions about all of the non-technical things that go along with being a software developer.',
  'score': 0.18999435531690817,
  'title': 'Soft Skills Engineering',
  'url': 'https://softskills.audio/'},
 {'description': "Lay yourself down to sleep with the soothing soporific of Miette\\'s purr as she reads you the world\\'s greatest works of short fiction, in a style all her own and in a way o

In [ ]:
def recs(genre_dict, genre_query):
    """Returns a list of recommendations based on interests user clicked in form
    Params: {query: list of genre names, genre_dict: dictionary that maps genre to titles, corr_dict: maps personality questions to answers given in survey}
    Returns: list of tuples containing titles and scores
    """

    genre_query=[genre.lower() for genre in genre_query]

    counter={}
    for cat in genre_query:
        for film in genre_dict[cat]:
            if film in counter:
                counter[film]+=1
            else:
                counter[film]=1
    results = list(counter.items())
    random.shuffle(results)  
    results.sort(key=lambda x: x[1], reverse=True)

    return results